In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import joblib
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

In [2]:
path = r"C:\Users\gagan\Downloads\Data\Winter 23-24\Capstone\fraud-transactions-detection\JPMorgan\Supervised\SupervisedData.csv"
df = pd.read_csv(filepath_or_buffer=path)
df.head()

,Transaction_Id,Sender_Id,Sender_Account,Sender_Country,Sender_Sector,Sender_lob,Bene_Id,Bene_Account,Bene_Country,USD_amount,Label,Transaction_Type,Date,Time
0,EXCHANGE-10115,JPMC-CLIENT-10098,ACCOUNT-10108,USA,35537,CCB,Unknown,Unknown,Unknown,558.43,0,WITHDRAWAL,2022-01-15,37440
1,QUICK-PAYMENT-10116,JPMC-CLIENT-10098,ACCOUNT-10109,USA,15287,CCB,CLIENT-10100,ACCOUNT-10106,CANADA,622.78,0,QUICK-PAYMENT,2022-01-15,37440
2,DEPOSIT-CASH-9833,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9812,ACCOUNT-9826,USA,802.54,0,DEPOSIT-CASH,2022-01-15,37440
3,PAY-CHECK-9832,JPMC-CLIENT-9812,ACCOUNT-9825,USA,38145,CCB,JPMC-CLIENT-9814,ACCOUNT-9824,USA,989.09,0,PAY-CHECK,2022-01-15,37440
4,DEPOSIT-CHECK-9806,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9789,ACCOUNT-9800,USA,786.78,0,DEPOSIT-CHECK,2022-01-15,37440


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2996354 entries, 0 to 2996353
Data columns (total 14 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Transaction_Id    object 
 1   Sender_Id         object 
 2   Sender_Account    object 
 3   Sender_Country    object 
 4   Sender_Sector     int64  
 5   Sender_lob        object 
 6   Bene_Id           object 
 7   Bene_Account      object 
 8   Bene_Country      object 
 9   USD_amount        float64
 10  Label             int64  
 11  Transaction_Type  object 
 12  Date              object 
 13  Time              int64  
dtypes: float64(1), int64(3), object(10)
memory usage: 320.0+ MB


In [4]:
df.describe()

,Sender_Sector,USD_amount,Label,Time
count,2.996354e+06,2.996354e+06,2.996354e+06,2.996354e+06
mean,2.136568e+04,5.134698e+02,2.057100e-02,4.315844e+04
std,1.599846e+04,5.398088e+02,1.419431e-01,2.489620e+04
min,0.000000e+00,0.000000e+00,0.000000e+00,2.400000e+02
25%,6.078000e+03,1.386900e+02,0.000000e+00,2.139500e+04
50%,2.075700e+04,4.041600e+02,0.000000e+00,4.347800e+04
75%,3.539000e+04,7.340700e+02,0.000000e+00,6.456200e+04
max,4.999900e+04,1.999894e+04,1.000000e+00,8.624400e+04


In [5]:
df.head()

,Transaction_Id,Sender_Id,Sender_Account,Sender_Country,Sender_Sector,Sender_lob,Bene_Id,Bene_Account,Bene_Country,USD_amount,Label,Transaction_Type,Date,Time
0,EXCHANGE-10115,JPMC-CLIENT-10098,ACCOUNT-10108,USA,35537,CCB,Unknown,Unknown,Unknown,558.43,0,WITHDRAWAL,2022-01-15,37440
1,QUICK-PAYMENT-10116,JPMC-CLIENT-10098,ACCOUNT-10109,USA,15287,CCB,CLIENT-10100,ACCOUNT-10106,CANADA,622.78,0,QUICK-PAYMENT,2022-01-15,37440
2,DEPOSIT-CASH-9833,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9812,ACCOUNT-9826,USA,802.54,0,DEPOSIT-CASH,2022-01-15,37440
3,PAY-CHECK-9832,JPMC-CLIENT-9812,ACCOUNT-9825,USA,38145,CCB,JPMC-CLIENT-9814,ACCOUNT-9824,USA,989.09,0,PAY-CHECK,2022-01-15,37440
4,DEPOSIT-CHECK-9806,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9789,ACCOUNT-9800,USA,786.78,0,DEPOSIT-CHECK,2022-01-15,37440


In [6]:
#adding the columns 
df["Date"] = pd.to_datetime(df["Date"])
df["Time"] = pd.to_datetime(df["Time"])
df["year"] = df["Date"].dt.year
df["month"] = df["Date"].dt.month
df["day"] = df["Date"].dt.day
df["hour"] = df["Time"].dt.hour
df["minute"] = df["Time"].dt.minute
df["second"] = df["Time"].dt.second
df["quarter"] = df["Date"].dt.quarter
df["day_of_week"] = df["Time"].dt.day_of_week
df["weekend"] = np.where(df["Time"].isin([5,6]),1,0)
df.drop(labels=["Time","Date"],axis=1,inplace=True)

In [7]:
df.head()

,Transaction_Id,Sender_Id,Sender_Account,Sender_Country,Sender_Sector,Sender_lob,Bene_Id,Bene_Account,Bene_Country,USD_amount,...,Transaction_Type,year,month,day,hour,minute,second,quarter,day_of_week,weekend
0,EXCHANGE-10115,JPMC-CLIENT-10098,ACCOUNT-10108,USA,35537,CCB,Unknown,Unknown,Unknown,558.43,...,WITHDRAWAL,2022,1,15,0,0,0,1,3,0
1,QUICK-PAYMENT-10116,JPMC-CLIENT-10098,ACCOUNT-10109,USA,15287,CCB,CLIENT-10100,ACCOUNT-10106,CANADA,622.78,...,QUICK-PAYMENT,2022,1,15,0,0,0,1,3,0
2,DEPOSIT-CASH-9833,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9812,ACCOUNT-9826,USA,802.54,...,DEPOSIT-CASH,2022,1,15,0,0,0,1,3,0
3,PAY-CHECK-9832,JPMC-CLIENT-9812,ACCOUNT-9825,USA,38145,CCB,JPMC-CLIENT-9814,ACCOUNT-9824,USA,989.09,...,PAY-CHECK,2022,1,15,0,0,0,1,3,0
4,DEPOSIT-CHECK-9806,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9789,ACCOUNT-9800,USA,786.78,...,DEPOSIT-CHECK,2022,1,15,0,0,0,1,3,0


In [8]:
df["Transaction_Id"].value_counts()

Transaction_Id
EXCHANGE-10115           2
PAYMENT-1464189          2
PAYMENT-1465967          2
MAKE-PAYMENT-1458992     2
PAYMENT-1462436          2
                        ..
PAYMENT-725385           2
PAYMENT-725382           2
DEPOSIT-CHECK-723150     2
PAY-BILL-725949          2
QUICK-PAYMENT-2181423    2
Name: count, Length: 1498177, dtype: int64

In [9]:
df.columns

Index(['Transaction_Id', 'Sender_Id', 'Sender_Account', 'Sender_Country',
       'Sender_Sector', 'Sender_lob', 'Bene_Id', 'Bene_Account',
       'Bene_Country', 'USD_amount', 'Label', 'Transaction_Type', 'year',
       'month', 'day', 'hour', 'minute', 'second', 'quarter', 'day_of_week',
       'weekend'],
      dtype='object')

In [10]:
for columns in df.columns:
    print(f"========{columns}========")
    print(df[columns].value_counts())
    
    

========Transaction_Id========
Transaction_Id
EXCHANGE-10115           2
PAYMENT-1464189          2
PAYMENT-1465967          2
MAKE-PAYMENT-1458992     2
PAYMENT-1462436          2
                        ..
PAYMENT-725385           2
PAYMENT-725382           2
DEPOSIT-CHECK-723150     2
PAY-BILL-725949          2
QUICK-PAYMENT-2181423    2
Name: count, Length: 1498177, dtype: int64
========Sender_Id========
Sender_Id
Unknown                 437772
JPMC-CLIENT-1730807        314
JPMC-CLIENT-1869649        310
JPMC-CLIENT-303611         310
JPMC-CLIENT-360470         308
                         ...  
BILL-COMPANY-1131040         2
BILL-COMPANY-1127400         2
BILL-COMPANY-1132967         2
BILL-COMPANY-1124250         2
BILL-COMPANY-2178781         2
Name: count, Length: 59072, dtype: int64
========Sender_Account========
Sender_Account
Unknown            437772
ACCOUNT-1591424       198
ACCOUNT-1829078       184
ACCOUNT-2088292       182
ACCOUNT-483507        182
                    

In [11]:
df.isna().sum()

Transaction_Id      0
Sender_Id           0
Sender_Account      0
Sender_Country      0
Sender_Sector       0
Sender_lob          0
Bene_Id             0
Bene_Account        0
Bene_Country        0
USD_amount          0
Label               0
Transaction_Type    0
year                0
month               0
day                 0
hour                0
minute              0
second              0
quarter             0
day_of_week         0
weekend             0
dtype: int64

In [12]:
df["sender_type"] = df["Sender_Id"].apply(lambda sender_id: "-".join(sender_id.split("-")[:-1]) if "-" in sender_id else sender_id)
df.drop(labels="Sender_Id",axis=1,inplace=True)

In [13]:
df["Bene_type"] = df["Bene_Id"].apply(lambda sender_id: "-".join(sender_id.split("-")[:-1]) if "-" in sender_id else sender_id)
df.drop(labels="Bene_Id",axis=1,inplace=True)

In [14]:
df["sender_type"].value_counts(),df["Bene_type"].value_counts()

(sender_type
 JPMC-CLIENT     2128122
 Unknown          437772
 CLIENT           221216
 COMPANY          118408
 JPMC-COMPANY      50026
 BILL-COMPANY      40810
 Name: count, dtype: int64,
 Bene_type
 JPMC-CLIENT     1045096
 BILL-COMPANY     724330
 CLIENT           614084
 Unknown          431786
 COMPANY          126968
 JPMC-COMPANY      54090
 Name: count, dtype: int64)

In [15]:
df.head()

,Transaction_Id,Sender_Account,Sender_Country,Sender_Sector,Sender_lob,Bene_Account,Bene_Country,USD_amount,Label,Transaction_Type,...,month,day,hour,minute,second,quarter,day_of_week,weekend,sender_type,Bene_type
0,EXCHANGE-10115,ACCOUNT-10108,USA,35537,CCB,Unknown,Unknown,558.43,0,WITHDRAWAL,...,1,15,0,0,0,1,3,0,JPMC-CLIENT,Unknown
1,QUICK-PAYMENT-10116,ACCOUNT-10109,USA,15287,CCB,ACCOUNT-10106,CANADA,622.78,0,QUICK-PAYMENT,...,1,15,0,0,0,1,3,0,JPMC-CLIENT,CLIENT
2,DEPOSIT-CASH-9833,Unknown,Unknown,0,Unknown,ACCOUNT-9826,USA,802.54,0,DEPOSIT-CASH,...,1,15,0,0,0,1,3,0,Unknown,JPMC-CLIENT
3,PAY-CHECK-9832,ACCOUNT-9825,USA,38145,CCB,ACCOUNT-9824,USA,989.09,0,PAY-CHECK,...,1,15,0,0,0,1,3,0,JPMC-CLIENT,JPMC-CLIENT
4,DEPOSIT-CHECK-9806,Unknown,Unknown,0,Unknown,ACCOUNT-9800,USA,786.78,0,DEPOSIT-CHECK,...,1,15,0,0,0,1,3,0,Unknown,JPMC-CLIENT


In [16]:
df.drop(["Bene_Account","Sender_Account","Transaction_Id"],axis=1,inplace=True)

In [17]:
df.columns

Index(['Sender_Country', 'Sender_Sector', 'Sender_lob', 'Bene_Country',
       'USD_amount', 'Label', 'Transaction_Type', 'year', 'month', 'day',
       'hour', 'minute', 'second', 'quarter', 'day_of_week', 'weekend',
       'sender_type', 'Bene_type'],
      dtype='object')

In [18]:
df["Sender_Sector"].value_counts()

Sender_Sector
0        437820
46269       100
47521        98
25097        94
37878        92
          ...  
18290        18
13613        18
44563        18
48632        16
20544        16
Name: count, Length: 50000, dtype: int64

In [19]:

Sender_lob_dummies  = pd.get_dummies(df["Sender_lob"],dtype="int")
df.drop("Sender_lob",axis=1,inplace=True)
df = df.assign(**Sender_lob_dummies)

In [20]:


Bene_Country_label_encoder = LabelEncoder()
df['Bene_Country_Encoded'] = Bene_Country_label_encoder.fit_transform(df['Bene_Country'])
df.drop("Bene_Country",axis=1,inplace=True)

Sender_Country_label_encoder = LabelEncoder()
df['Sender_Country_Encoded'] = Sender_Country_label_encoder.fit_transform(df['Sender_Country'])
df.drop("Sender_Country",axis=1,inplace=True)

Transaction_Type_label_encoder = LabelEncoder()
df['Transaction_Type_Encoded'] = Transaction_Type_label_encoder.fit_transform(df['Transaction_Type'])
df.drop("Transaction_Type",axis=1,inplace=True)

sender_type_label_encoder = LabelEncoder()
df['sender_type_Encoded'] = sender_type_label_encoder.fit_transform(df['sender_type'])
df.drop("sender_type",axis=1,inplace=True)

Bene_type_label_encoder = LabelEncoder()
df['Bene_type_Encoded'] = Bene_type_label_encoder.fit_transform(df['Bene_type'])
df.drop("Bene_type",axis=1,inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2996354 entries, 0 to 2996353
Data columns (total 19 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Sender_Sector             int64  
 1   USD_amount                float64
 2   Label                     int64  
 3   year                      int32  
 4   month                     int32  
 5   day                       int32  
 6   hour                      int32  
 7   minute                    int32  
 8   second                    int32  
 9   quarter                   int32  
 10  day_of_week               int32  
 11  weekend                   int32  
 12  CCB                       int32  
 13  Unknown                   int32  
 14  Bene_Country_Encoded      int32  
 15  Sender_Country_Encoded    int32  
 16  Transaction_Type_Encoded  int32  
 17  sender_type_Encoded       int32  
 18  Bene_type_Encoded         int32  
dtypes: float64(1), int32(16), int64(2)
memory usage: 251.5 MB


In [22]:
#didviding the data 
X_data = df.drop(["Label"],axis=1)
Y_data =  df["Label"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2097447, 18), (898907, 18), (2097447,), (898907,))

In [23]:
#implementing the   decision tree model sklearn.tree.DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

# Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Decision Tree Classifer Accuracy:", accuracy_score(y_test, y_pred))

Decision Tree Classifer Accuracy: 0.9874536520463185


In [24]:
# # Support Vector Machine (SVM) Classifier
# from sklearn.svm import SVC
# svm_clf = SVC()
# svm_clf.fit(X_train, y_train)
# svm_pred = svm_clf.predict(X_test)
# svm_accuracy = accuracy_score(y_test, svm_pred)
# print("SVM Accuracy:", svm_accuracy)

In [25]:

# Random Forest Classifier

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
print("Random Forest Accuracy:", rf_accuracy)

Random Forest Accuracy: 0.9936534035222776


In [26]:

rf_classification_report = classification_report(y_test, rf_pred)
print(rf_classification_report)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00    880384
           1       0.98      0.71      0.82     18523

    accuracy                           0.99    898907
   macro avg       0.99      0.85      0.91    898907
weighted avg       0.99      0.99      0.99    898907



In [27]:
# K-Nearest Neighbors (KNN) Classifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)
knn_pred = knn_clf.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_pred)
print("KNN Accuracy:", knn_accuracy)

KNN Accuracy: 0.9795440462695251


In [28]:
knn_classification_report = classification_report(y_test, knn_pred)
print(knn_classification_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    880384
           1       0.54      0.05      0.10     18523

    accuracy                           0.98    898907
   macro avg       0.76      0.53      0.54    898907
weighted avg       0.97      0.98      0.97    898907



In [29]:
# Logistic Regression Classifier
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
lr_accuracy = accuracy_score(y_test, lr_pred)
print("Logistic Regression Accuracy:", lr_accuracy)
lr_accuracy = accuracy_score(y_test, lr_pred)
lr_precision = precision_score(y_test, lr_pred, average='weighted')
lr_recall = recall_score(y_test, lr_pred, average='weighted')
lr_f1 = f1_score(y_test, lr_pred, average='weighted')
print("\nLR Metrics:")
print("Accuracy:", lr_accuracy)
print("Precision:", lr_precision)
print("Recall:", lr_recall)
print("F1 Score:", lr_f1)

c:\Users\gagan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.9797220402110564


NameError: name '_pred' is not defined

In [30]:
lr_classification_report = classification_report(y_test, lr_pred)
print(lr_classification_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    880384
           1       0.91      0.02      0.03     18523

    accuracy                           0.98    898907
   macro avg       0.95      0.51      0.51    898907
weighted avg       0.98      0.98      0.97    898907



In [31]:
# Multi-Layer Perceptron (MLP) Classifier
mlp_clf = MLPClassifier()
mlp_clf.fit(X_train, y_train)
mlp_pred = mlp_clf.predict(X_test)
mlp_accuracy = accuracy_score(y_test, mlp_pred)
mlp_precision = precision_score(y_test, mlp_pred, average='weighted')
mlp_recall = recall_score(y_test, mlp_pred, average='weighted')
mlp_f1 = f1_score(y_test, mlp_pred, average='weighted')
print("\nMLP Metrics:")
print("Accuracy:", mlp_accuracy)
print("Precision:", mlp_precision)
print("Recall:", mlp_recall)
print("F1 Score:", mlp_f1)


MLP Metrics:
Accuracy: 0.9799078213875295
Precision: 0.9790916024489877
Recall: 0.9799078213875295
F1 Score: 0.9705181806279726


In [32]:
mlp_classification_report = classification_report(y_test, mlp_pred)
print(mlp_classification_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    880384
           1       0.94      0.03      0.05     18523

    accuracy                           0.98    898907
   macro avg       0.96      0.51      0.52    898907
weighted avg       0.98      0.98      0.97    898907



In [33]:
# AdaBoost Classifier
ada_clf = AdaBoostClassifier()
ada_clf.fit(X_train, y_train)
ada_pred = ada_clf.predict(X_test)
ada_accuracy = accuracy_score(y_test, ada_pred)
ada_precision = precision_score(y_test, ada_pred, average='weighted')
ada_recall = recall_score(y_test, ada_pred, average='weighted')
ada_f1 = f1_score(y_test, ada_pred, average='weighted')
print("\nAdaBoost Metrics:")
print("Accuracy:", ada_accuracy)
print("Precision:", ada_precision)
print("Recall:", ada_recall)
print("F1 Score:", ada_f1)

c:\Users\gagan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



AdaBoost Metrics:
Accuracy: 0.9800135052903137
Precision: 0.9785549605588745
Recall: 0.9800135052903137
F1 Score: 0.9708332450910528


In [34]:
ada_classification_report = classification_report(y_test, ada_pred)
print(ada_classification_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    880384
           1       0.91      0.03      0.06     18523

    accuracy                           0.98    898907
   macro avg       0.94      0.52      0.53    898907
weighted avg       0.98      0.98      0.97    898907



In [35]:
# Naive Bayes Classifier (Gaussian)
nb_clf = GaussianNB()
nb_clf.fit(X_train, y_train)
nb_pred = nb_clf.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_pred)
nb_precision = precision_score(y_test, nb_pred, average='weighted')
nb_recall = recall_score(y_test, nb_pred, average='weighted')
nb_f1 = f1_score(y_test, nb_pred, average='weighted')
print("\nNaive Bayes Metrics:")
print("Accuracy:", nb_accuracy)
print("Precision:", nb_precision)
print("Recall:", nb_recall)
print("F1 Score:", nb_f1)


Naive Bayes Metrics:
Accuracy: 0.9800135052903137
Precision: 0.9785549605588745
Recall: 0.9800135052903137
F1 Score: 0.9708332450910528


In [36]:
nb_classification_report = classification_report(y_test, nb_pred)
print(nb_classification_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    880384
           1       0.91      0.03      0.06     18523

    accuracy                           0.98    898907
   macro avg       0.94      0.52      0.53    898907
weighted avg       0.98      0.98      0.97    898907



In [37]:
# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
gb_precision = precision_score(y_test, gb_pred, average='weighted')
gb_recall = recall_score(y_test, gb_pred, average='weighted')
gb_f1 = f1_score(y_test, gb_pred, average='weighted')
print("\nGradient Boosting Metrics:")
print("Accuracy:", gb_accuracy)
print("Precision:", gb_precision)
print("Recall:", gb_recall)
print("F1 Score:", gb_f1)


Gradient Boosting Metrics:
Accuracy: 0.9800368669951397
Precision: 0.9793050139127066
Recall: 0.9800368669951397
F1 Score: 0.9708343895589243


In [38]:
gb_classification_report = classification_report(y_test, gb_pred)
print(gb_classification_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    880384
           1       0.94      0.03      0.06     18523

    accuracy                           0.98    898907
   macro avg       0.96      0.52      0.53    898907
weighted avg       0.98      0.98      0.97    898907



In [ ]:

# Save the trained models
joblib.dump(clf, 'decision_tree_model.pkl')
joblib.dump(rf_clf, 'random_forest_model.pkl')
joblib.dump(knn_clf, 'knn_model.pkl')
joblib.dump(lr_clf, 'logistic_regression_model.pkl')
joblib.dump(mlp_clf, 'mlp_model.pkl')
joblib.dump(ada_clf, 'adaboost_model.pkl')
joblib.dump(nb_clf, 'naive_bayes_model.pkl')
joblib.dump(gb_clf, 'gradient_boosting_model.pkl')

# Load the saved models
loaded_decision_tree_model = joblib.load('decision_tree_model.pkl')
loaded_random_forest_model = joblib.load('random_forest_model.pkl')
loaded_knn_model = joblib.load('knn_model.pkl')
loaded_logistic_regression_model = joblib.load('logistic_regression_model.pkl')
loaded_mlp_model = joblib.load('mlp_model.pkl')
loaded_adaboost_model = joblib.load('adaboost_model.pkl')
loaded_naive_bayes_model = joblib.load('naive_bayes_model.pkl')
loaded_gradient_boosting_model = joblib.load('gradient_boosting_model.pkl')

# Test the loaded models
# Assuming X_test and y_test are already defined

# Decision Tree
y_pred_dt = loaded_decision_tree_model.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy:", accuracy_dt)

# Random Forest
y_pred_rf = loaded_random_forest_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)

# KNN
y_pred_knn = loaded_knn_model.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("KNN Accuracy:", accuracy_knn)

# Logistic Regression
y_pred_lr = loaded_logistic_regression_model.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Logistic Regression Accuracy:", accuracy_lr)

# MLP
y_pred_mlp = loaded_mlp_model.predict(X_test)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print("MLP Accuracy:", accuracy_mlp)

# AdaBoost
y_pred_ada = loaded_adaboost_model.predict(X_test)
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print("AdaBoost Accuracy:", accuracy_ada)

# Naive Bayes
y_pred_nb = loaded_naive_bayes_model.predict(X_test)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print("Naive Bayes Accuracy:", accuracy_nb)

# Gradient Boosting
y_pred_gb = loaded_gradient_boosting_model.predict(X_test)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Gradient Boosting Accuracy:", accuracy_gb)


In [ ]:
# # Assuming X_train, X_test, y_train, and y_test are already defined

# # Create and fit the Isolation Forest model
# if_clf = IsolationForest(random_state=42)
# if_clf.fit(X_train)

# # Predict anomalies (-1) or normal instances (1)
# if_pred = if_clf.predict(X_test)

# # Convert predictions to binary labels for evaluation
# # Assuming anomalies are labeled as -1 and normal instances as 1
# if_pred_binary = [1 if x == 1 else 0 for x in if_pred]

# # Calculate evaluation metrics
# if_accuracy = accuracy_score(y_test, if_pred_binary)
# if_precision = precision_score(y_test, if_pred_binary)
# if_recall = recall_score(y_test, if_pred_binary)
# if_f1 = f1_score(y_test, if_pred_binary)

# print("\nIsolation Forest Metrics:")
# print("Accuracy:", if_accuracy)
# print("Precision:", if_precision)
# print("Recall:", if_recall)
# print("F1 Score:", if_f1)


In [ ]:
# from sklearn.ensemble import IsolationForest
# from sklearn.neighbors import LocalOutlierFactor
# from sklearn.metrics import classification_report,accuracy_score


In [ ]:
# classifiers = {
#     "Isolation Forest":IsolationForest(n_estimators=100, max_samples=len(X_train),
#                                        contamination=0.02,random_state=42, verbose=0),
#     "Local Outlier Factor":LocalOutlierFactor(n_neighbors=20, algorithm='auto',
#                                               leaf_size=30, metric='minkowski',
#                                               p=2, metric_params=None, contamination=0.02),

# }

In [ ]:
# n_outliers = len(Fraud)
# for i, (clf_name,clf) in enumerate(classifiers.items()):
#     #Fit the data and tag outliers
#     if clf_name == "Local Outlier Factor":
#         y_pred = clf.fit_predict(X)
#         scores_prediction = clf.negative_outlier_factor_
#     else:
#         clf.fit(X)
#         scores_prediction = clf.decision_function(X)
#         y_pred = clf.predict(X)
#     #Reshape the prediction values to 0 for Valid transactions , 1 for Fraud transactions
#     y_pred[y_pred == 1] = 0
#     y_pred[y_pred == -1] = 1
#     n_errors = (y_pred != Y).sum()
#     # Run Classification Metrics
#     print("{}: {}".format(clf_name,n_errors))
#     print("Accuracy Score :")
#     print(accuracy_score(Y,y_pred))
#     print("Classification Report :")
#     print(classification_report(Y,y_pred))*